# Intent Detection

## Initial Setup

In [3]:
import sys
import os
import datetime
import pandas as pd

notebook_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(notebook_dir, "..", ".."))
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

from scripts import test_config
from src.utils import parse, common, handlebars, generate, intent

config_data = test_config.get_config_data()

correct_predictions = 0
incorrect_predictions = 0
incorrect_span_ids = []

system_prompt_path = config_data["intent_detection"]["system_prompt_path"]
user_prompt_path = config_data["intent_detection"]["user_prompt_path"]
dataset_path = config_data["intent_detection"]["intent_detection_test_dataset"]

model = config_data["intent_detection"]["model"]
temperature = float(config_data["intent_detection"]["model_temperature"])

print("model:", model)
print("temperature:", temperature)

model: gpt-4o
temperature: 1.0


## Load the system prompt

In [4]:
system_prompt_content = common.read_file(os.path.join(parent_dir, system_prompt_path))

if system_prompt_content is None:
    print("No system prompt content found.")
    exit()

print("System prompt content:", system_prompt_content)

System prompt content: You are a message classifier that reviews chat histories and decides whether the LAST MESSAGE is requesting an image.

When an image is requested, you extract additional metadata about the requested image. Your responses must always adhere to the following json schema:

{
    "$schema": "https://json-schema.org/draft/2020-12/schema",
    "type": "object",
    "properties": {
        "send_image": {
            "type": "boolean",
            "description": "true if the LAST MESSAGE in the chat is requesting an image from a bot. If the LAST MESSAGE does not request an image from a bot, the value should be false and no other metadata included."
        },
        "user_requestor": {
            "type": "string",
            "description": "Name of the human user who is asking for an image, if any. Only real users can be a user_requestor. If a user asks for an amendment to an image, the user_requestor is the user asking for the amendment."
        },
        "bot_sen

## Load the user prompt

In [5]:
user_prompt_content = common.read_file(os.path.join(parent_dir, user_prompt_path))

if user_prompt_content is None:
    print("No user prompt content found.")
    exit()

print("User prompt content:", user_prompt_content)

User prompt content: Determine whether the LAST MESSAGE in this chat history is requesting an image.

CHAT HISTORY:
{{chat_history}}

LAST MESSAGE:
{{last_message}}

OUTPUT:



## Load the dataset

In [7]:
csv_data = common.read_csv_file(os.path.join(parent_dir, dataset_path))
if csv_data is None:
    print("No Dataset data found.")
    exit()

df = pd.DataFrame(csv_data)

df.head()


,span_id,model,user_prompt,response,annotation,notes
0,28c1d17108f3b6fe,gpt-4o,Determine whether the LAST MESSAGE in this cha...,"```json\n{\n ""send_image"": true,\n ""user_reque...",TP,
1,6475d9729ef63ca8,gpt-4o,Determine whether the LAST MESSAGE in this cha...,"{\n ""send_image"": true,\n ""user_requestor"": ""d...",TP,
2,83e0aa2d71ffde31,gpt-4o,Determine whether the LAST MESSAGE in this cha...,"{\n ""send_image"": true,\n ""user_requestor"": ""a...",TP,
3,3342fbbb99e69a2f,gpt-4o,Determine whether the LAST MESSAGE in this cha...,"{\n ""send_image"": true,\n ""user_requestor"": ""c...",TP,
4,c17f8cd72e8b83ca,gpt-4o,Determine whether the LAST MESSAGE in this cha...,"{\n ""send_image"": true,\n ""user_requestor"": ""c...",TP,


## Evaluate model

In [8]:
for i in range(len(csv_data)):
    messages = intent.extract_messages(csv_data[i]['user_prompt'])
    chat_history_only = messages['chat_history']
    last_message = messages['last_message']

    user_prompt_data = {'chat_history': chat_history_only, 'last_message': last_message}

    user_prompt_rendered = handlebars.compile_prompt(user_prompt_content, user_prompt_data)

    if user_prompt_rendered is None:
        print("No user prompt content rendered.")
        exit()

    messages = [
        {"role": "system", "content": system_prompt_content},
        {"role": "user", "content": user_prompt_rendered},
    ]

    intent_response = generate.get_simple_completion(config_data, messages, model, temperature)

    # Canned response to avoid calling the API
#    intent_response = """{
#"send_image": false
#}"""

    try:
        isCorrect = intent.evaluate_response(intent_response, csv_data[i]['response'])

        if (isCorrect):
            correct_predictions += 1
        else:
            incorrect_predictions += 1
            incorrect_span_ids.append(csv_data[i]['span_id'])

        csv_data[i]['new_model'] = model
        csv_data[i]['new_response'] = intent_response
        csv_data[i]['is_correct'] = isCorrect
    except ValueError as e:
        print(f"\nWarning: evaluating response failed: {str(e)} on span_id {csv_data[i]['span_id']}")

    print(".", end="")

.............................................................

## Generate and Save Report

In [9]:
report = intent.generate_report(correct_predictions, incorrect_predictions, incorrect_span_ids)
print(report)

timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
report_path = f"report_{timestamp}.txt"

common.save_file(report_path, report)

Correct predictions: 53
Incorrect predictions: 8
Accuracy: 86.89%
Incorrect span IDs:
- 5bcf01044aede5e9
- 410f60d34e670e6d
- 322f641426de8ea2
- 540efdae25d0d706
- 9cb20d1a706f11fb
- 6fd1bf74fd6c54de
- 1a8ce592e1f1601d
- fe3a64e4805dd270


File report_20241018_140809.txt written successfully.


## Save Evaluation to CSV

In [10]:
evaluation_output_path = f"evaluation_{timestamp}.csv"
common.save_csv_file(evaluation_output_path, csv_data)


File evaluation_20241018_140809.csv written successfully.
